# ДОМАШНЕЕ ЗАДАНИЕ № 1. Spark RDD API
---
**Дисциплина:** Методы обработки больших данных  
**Студент:** Голдышев Д.М. (goldyshev02@mail.ru)  
**Группа:** ИУ6-31М  

## Задание 3. Косинусное сходство между фильмами
---
### Цель
Вычислить косинусное сходство между рейтингами фильмов, используя **только стандартные операции RDD API**.

### Задачи
1. Вычислить косинусное сходство между рейтингами фильмов.
2. Для фильма с `movieId = 589` сформировать RDD со значениями сходства с остальными фильмами.
3. Добавить наименования фильмов.
4. Вывести топ-10 наиболее похожих фильмов.

### Данные
Используется датасет [MovieLens (ml-latest-small)](http://files.grouplens.org/datasets/movielens/ml-latest-small.zip):
- `ratings.csv` — рейтинги пользователей (userId, movieId, rating, timestamp)
- `movies.csv` — справочник фильмов (movieId, title, genres)

## Шаг 1. Импорт зависимостей и конфигурация
---

In [1]:
import os
import sys
import errno
import tempfile
from pathlib import Path

import math
import csv
import io
from pyspark import SparkContext, SparkConf 

TARGET_MOVIE_ID = 589 # ID фильма, для которого ищем похожие
TOP_N = 10 # Количество похожих фильмов для вывода

DATA_PATH = "/app/data/ml-latest-small" # Путь к директории с данными MovieLens
RATINGS_PATH = f"{DATA_PATH}/ratings.csv" # Путь к данным с рейтингами
MOVIES_PATH = f"{DATA_PATH}/movies.csv" # Путь к данным с фильмами
OUTPUT_PATH = "./hw1_rdd_cosine_results.csv" # Путь для сохранения результатов

MIN_COMMON_USERS_INITIAL = 10 # Начальный порог минимального количества общих пользователей
MIN_COMMON_USERS_FLOOR = 2 # Минимально допустимый порог
THRESHOLD_STEP = 2 # Шаг уменьшения порога при fallback

### Проверка существования данных

In [2]:
def check_file(path: str):
    if not os.path.isfile(path):
        raise FileNotFoundError(f"{path}: файл не найден")
    try:
        with open(path, "r", encoding="utf-8") as f:
            f.read(1)  # пробное чтение
    except Exception as e:
        raise IOError(f"{path}: файл существует, но не читается: {e}")

try:
    check_file(RATINGS_PATH)
    check_file(MOVIES_PATH)
    print("Файлы, необходимые для работы, найдены и читаются.")
except Exception as e:
    print("Ошибка при проверке файлов:")
    print(e)
    raise SystemExit(1)

Файлы, необходимые для работы, найдены и читаются.


### Проверка возможности сохранения результатов

In [3]:
def ensure_output_path(path: str) -> Path:
    p = Path(path).expanduser()
    if p.exists() and p.is_dir():
        dir_path = p
    else:
        if str(p).endswith(os.sep) or (not p.exists() and p.suffix == ""):
            dir_path = p
        else:
            dir_path = p.parent
    if not str(dir_path):
        dir_path = Path(".")
    try:
        dir_path.mkdir(parents=True, exist_ok=True)
    except PermissionError as e:
        raise PermissionError(f"{dir_path}: нет прав на создание директории: {e}")
    except OSError as e:
        raise OSError(f"{dir_path}: не удалось создать директорию: {e}")
    if not os.access(dir_path, os.W_OK):
        raise PermissionError(f"{dir_path}: нет прав на запись в директорию")
    try:
        with tempfile.NamedTemporaryFile(dir=dir_path, delete=True) as tmp:
            tmp.write(b"test")
            tmp.flush()
    except OSError as e:
        raise OSError(f"{dir_path}: невозможно записать временный файл: {e}")
    return dir_path

try:
    ensure_output_path(OUTPUT_PATH)
    print("Можно сохранять результаты в указанные директории")
except Exception as e:
    print("Ошибка при проверке директорий для записи результатов:")
    print(e)
    raise SystemExit(1)

Можно сохранять результаты в указанные директории


## Шаг 2. Функции парсинга данных
---

In [4]:
def parse_rating(line):
    """
    Парсинг строки ratings.csv -> (userId, movieId, rating)
    """
    reader = csv.reader(io.StringIO(line))
    parts = next(reader)
    # Возвращаем userId, movieId, rating (игнорируем parts[3] - timestamp)
    return (int(parts[0]), int(parts[1]), float(parts[2]))

def parse_movie(line):
    """
    Парсинг строки movies.csv -> (movieId, title)
    """
    reader = csv.reader(io.StringIO(line))
    parts = next(reader)
    # Возвращаем movieId, title (игнорируем parts[2] - genres)
    return (int(parts[0]), parts[1])

## Шаг 3. Инициализация SparkContext
---
SparkContext управляет подключением к кластеру Spark и является обязательным для любых RDD-операций.

In [5]:
sc = SparkContext.getOrCreate(
    SparkConf()
        .setAppName("HW1_CosineSimilarity_RDD")
        .setMaster("local[*]")
)
sc

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/12/10 23:17:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<SparkContext master=local[*] appName=HW1_CosineSimilarity_RDD>

## Шаг 4. Загрузка и предобработка данных
---

In [6]:
# Загрузка рейтингов
ratings_raw = sc.textFile(RATINGS_PATH)
header_ratings = ratings_raw.first()
ratings_rdd = ratings_raw \
    .filter(lambda line: line != header_ratings) \
    .map(parse_rating) \
    .cache() # Cохраняем RDD в памяти для повторного использования

# Загрузка фильмов
movies_raw = sc.textFile(MOVIES_PATH)
header_movies = movies_raw.first()
movies_rdd = movies_raw \
    .filter(lambda line: line != header_movies) \
    .map(parse_movie)

# Собираем словарь фильмов (т.к. датасет небольшой)
movies_dict = dict(movies_rdd.collect())
# Создаём broadcast-переменную для передачи на все worker-узлы
movies_broadcast = sc.broadcast(movies_dict)

# Выводим статистику загрузки
print(f"Загружено рейтингов: {ratings_rdd.count()}")
print(f"Загружено фильмов: {len(movies_dict)}")

Загружено рейтингов: 100836
Загружено фильмов: 9742


## Шаг 5. Подготовка данных о рейтингах пользователей
---

In [7]:
user_movie = ratings_rdd.map(
    lambda x: (
        x[0],          # ключ: userId
        (x[1], x[2])   # значение: (movieId, rating)
    )
)

## Шаг 6. Генерация пар фильмов
---

In [8]:
# self-join по userId, чтобы получить пары фильмов, оценённых одним пользователем
movie_pairs = (
    user_movie
        # join по ключу userId
        .join(user_movie)
        # Убираем дубликаты пар фильмов
        .filter(lambda x: x[1][0][0] < x[1][1][0])
        # Приводим к формату: ((movie1, movie2), (rating1, rating2))
        .map(lambda x: (
            # ключ: пара идентификаторов фильмов (movie1, movie2)
            (x[1][0][0], x[1][1][0]),
            # значение: соответствующие оценки (rating1, rating2)
            (x[1][0][1], x[1][1][1])
        ))
)

## Шаг 7. Вычисление косинусного сходства
---
#### Формула косинусного сходства
Для двух фильмов A и B:
$$
\text{sim}(A, B) = \frac{\sum_{u} r_{uA} \cdot r_{uB}}{\sqrt{\sum_{u} r_{uA}^2} \cdot \sqrt{\sum_{u} r_{uB}^2}}
$$
где $r_{uA}$ — рейтинг пользователя $u$ для фильма $A$, суммирование по пользователям, оценившим оба фильма.

In [9]:
# Начальное значение аккумулятора: (dot_product, norm1_sq, norm2_sq, count)
# dot_product — скалярное произведение векторов рейтингов
# norm1_sq — сумма квадратов рейтингов первого фильма
# norm2_sq — сумма квадратов рейтингов второго фильма  
# count — количество общих пользователей
ZERO_VALUE = (0.0, 0.0, 0.0, 0)

def seq_op(acc, val):
    """
    Обновление аккумулятора новым значением пары рейтингов.
    Args:
        acc: текущий аккумулятор (dot, norm1_sq, norm2_sq, count)
        val: новое значение (rating1, rating2)
    Returns:
        обновлённый аккумулятор
    """
    r1, r2 = val
    return (
        acc[0] + r1 * r2,      # dot_product += r1 * r2
        acc[1] + r1 * r1,      # norm1_sq += r1^2
        acc[2] + r2 * r2,      # norm2_sq += r2^2
        acc[3] + 1             # count += 1
    )

def comb_op(acc1, acc2):
    """
    Объединяет два частичных результата.
    Args:
        acc1, acc2: два аккумулятора для объединения
    Returns:
        объединённый аккумулятор
    """
    return (
        acc1[0] + acc2[0],  # суммируем dot_product
        acc1[1] + acc2[1],  # суммируем norm1_sq
        acc1[2] + acc2[2],  # суммируем norm2_sq
        acc1[3] + acc2[3]   # суммируем count
    )


def compute_final_similarity(acc):
    """
    Вычисление финального значения косинусного сходства из аккумулятора.
    Args:
        acc: финальный аккумулятор
    Returns:
        (similarity, count) [или (0.0, count) при делении на ноль]
    """
    dot, norm1_sq, norm2_sq, count = acc
    
    # Защита от деления на ноль
    if count == 0 or norm1_sq == 0 or norm2_sq == 0:
        return (0.0, count)
    
    # Косинусное сходство = dot / (||v1|| * ||v2||)
    similarity = dot / (math.sqrt(norm1_sq) * math.sqrt(norm2_sq))
    return (similarity, count)


# Вычисление сходства
similarities = movie_pairs \
    .aggregateByKey(ZERO_VALUE, seq_op, comb_op) \
    .mapValues(compute_final_similarity) \
    .map(lambda x: (x[0][0], x[0][1], x[1][0], x[1][1])) \
    .cache()
# Структура результата: (movie1, movie2, similarity, common_users_count)

print(f"Вычислено пар сходства: {similarities.count()}")
print("Первые 10 пар сходства:")
similarities.take(10)

Вычислено пар сходства: 13157672
Первые 10 пар сходства:


25/12/10 23:19:38 WARN BlockManager: Task 18 already completed, not releasing lock for rdd_20_0


[(125, 357, 0.9116377679037143, 7),
 (162, 2336, 0.9307253886204377, 7),
 (588, 3386, 0.9787962339515168, 15),
 (898, 3044, 0.9938837346736188, 2),
 (902, 904, 0.9873303331142156, 17),
 (937, 2145, 0.9978801059658184, 2),
 (1198, 1892, 0.9595213389890941, 8),
 (1199, 1895, 0.9070543032310491, 4),
 (2843, 4239, 0.9952226187595397, 3),
 (260, 4878, 0.9524073755949048, 72)]

## Шаг 8. Фильтрация для целевого фильма
---
При вычислении косинусного сходства между двумя фильмами важно учитывать **статистическую надёжность оценки**. Если два фильма имеют всего несколько общих пользователей, то их косинусное сходство может быть случайным и не отражать реальной похожести. Например, если оба фильма оценили только 2 человека, и оба поставили одинаковые оценки, сходство будет равно 1.0 — но это совершенно не означает, что фильмы действительно похожи.

Поэтому, для повышения статистической значимости сходства и для предотвращения попадания в формируемый топ случайно высоких коэффициентов, было решено **формировать результирующий список похожих фильмов только из пар, где количество общих пользователей превышает заданный порог**. В случае недостатка результатов порог постепенно снижается, но остаётся в пределах "разумного минимума", гарантирующего хотя бы базовую достоверность результатов.

In [10]:
def get_similar_movies(similarities_rdd, target_id, min_common, top_n):
    """
    Фильтрует и возвращает топ похожих фильмов для целевого фильма.
    Args:
        similarities_rdd: RDD с (movie1, movie2, similarity, count)
        target_id: ID целевого фильма
        min_common: минимальное количество общих пользователей
        top_n: количество результатов
    Returns:
        список [(other_movie_id, similarity, common_count), ...]
    """
    # Фильтруем пары, где участвует целевой фильм
    filtered = similarities_rdd \
        .filter(lambda x: x[0] == target_id or x[1] == target_id) \
        .map(lambda x: (
            # Извлекаем ID похожего фильма
            x[1] if x[0] == target_id else x[0],
            x[2],  # similarity
            x[3]   # common_users_count
        )) \
        .filter(lambda x: x[2] >= min_common)  # Фильтр по минимуму общих пользователей
    
    # Сортируем по убыванию сходства и берём top_n
    return filtered.sortBy(lambda x: -x[1]).take(top_n)

current_threshold = MIN_COMMON_USERS_INITIAL
top_similar = []

print(f"\nПоиск похожих фильмов для movieId = {TARGET_MOVIE_ID}")
print("-" * 40)

# Уменьшаем порог, пока не найдём достаточно результатов
while current_threshold >= MIN_COMMON_USERS_FLOOR:
    print(f"Попытка с порогом общих пользователей >= {current_threshold}...")
    top_similar = get_similar_movies(
        similarities, 
        TARGET_MOVIE_ID, 
        current_threshold, 
        TOP_N
    )
    if len(top_similar) >= TOP_N:
        # Нашли достаточно результатов
        print(f"Найдено {len(top_similar)} фильмов")
        break
    else:
        # Не хватает результатов — уменьшаем порог
        print(f"  Найдено только {len(top_similar)}, уменьшаем порог...")
        current_threshold -= THRESHOLD_STEP

# Проверка, если даже с минимальным порогом недостаточно результатов
if len(top_similar) < TOP_N:
    print(f"Внимание: найдено только {len(top_similar)} похожих фильмов")
    print(f"(даже с минимальным порогом {MIN_COMMON_USERS_FLOOR})")

# Получаем название целевого фильма
movies_local = movies_broadcast.value
target_title = movies_local.get(TARGET_MOVIE_ID, "Unknown")

print(f"\nИспользуемый порог: {current_threshold} общих пользователей")


Поиск похожих фильмов для movieId = 589
----------------------------------------
Попытка с порогом общих пользователей >= 10...


[Stage 18:>                                                         (0 + 4) / 4]

Найдено 10 фильмов

Используемый порог: 10 общих пользователей


## Шаг 9. Вывод и сохранение результатов
---

In [11]:
print("=" * 70)
print(f"Топ-{len(top_similar)} фильмов, похожих на:")
print(f"  [{TARGET_MOVIE_ID}] {target_title}")
print("=" * 70)
print(f"{'Rank':<5} {'MovieID':<8} {'Similarity':<12} {'Common':<8} Title")
print("-" * 70)

# Список для сохранения в файл
results = []

# Выводим результаты в консоль
for rank, (movie_id, sim, count) in enumerate(top_similar, 1):
    title = movies_local.get(movie_id, "Unknown")
    print(f"{rank:<5} {movie_id:<8} {sim:<12.6f} {count:<8} {title}")
    results.append(f"{rank},{movie_id},{sim:.6f},{count},{title}")

# Сохраняем результаты в файл
with open(OUTPUT_PATH, "w", encoding="utf-8") as f:    
    # Заголовок CSV
    f.write("rank,movieId,similarity,common_users,title\n")
    # Записываем все строки результатов
    for line in results:
        f.write(line + "\n")

print(f"\nРезультаты сохранены: {OUTPUT_PATH}")

Топ-10 фильмов, похожих на:
  [589] Terminator 2: Judgment Day (1991)
Rank  MovieID  Similarity   Common   Title
----------------------------------------------------------------------
1     1223     0.995822     11       Grand Day Out with Wallace and Gromit, A (1989)
2     82461    0.995054     11       Tron: Legacy (2010)
3     7482     0.994651     12       Enter the Dragon (1973)
4     119145   0.994354     17       Kingsman: The Secret Service (2015)
5     33660    0.993120     10       Cinderella Man (2005)
6     6586     0.992872     10       American Wedding (American Pie 3) (2003)
7     27831    0.992559     14       Layer Cake (2004)
8     1231     0.992442     12       Right Stuff, The (1983)
9     86190    0.991156     10       Hanna (2011)
10    4865     0.989886     12       From Hell (2001)

Результаты сохранены: ./hw1_rdd_cosine_results.csv


## Шаг 10. Остановка SparkContext
---

In [12]:
sc.stop()

## Выводы
---
В рамках задания было вычислено косинусное сходство между фильмами MovieLens с использованием чистого **Spark RDD API**. Построены пары фильмов, оценённых одними и теми же пользователями, рассчитаны необходимые статистики и получено сходство для всех комбинаций. Для заданного фильма **сформирован список наиболее похожих фильмов** с добавлением их названий.

Для повышения качества результатов применён фильтр по минимальному количеству общих пользователей. Это исключает пары, где высокое сходство возникает случайно из-за 1–2 совпавших оценок, и обеспечивает **статистическую надёжность** итогового топа.

**Итоговый топ-10 похожих фильмов успешно получен, отсортирован и сохранён.**